In [1]:
import pandas as pd
import time 
from datetime import datetime as dt
import yfinance as yf

In [2]:
tick = yf.Ticker("GOOGL")
# get stock info

# get historical market data
hist = tick.history(period="6y")
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2014-07-22,599.70,608.89,599.26,603.57,1881600,0,0
2014-07-23,602.05,607.05,601.35,605.19,1106000,0,0
2014-07-24,605.53,608.91,601.00,603.01,1355000,0,0
2014-07-25,599.25,601.00,595.75,598.08,1360000,0,0
2014-07-28,597.75,601.10,592.59,599.02,1358100,0,0
...,...,...,...,...,...,...,...
2020-07-16,1501.50,1515.91,1488.27,1514.92,1324200,0,0
2020-07-17,1516.91,1523.43,1498.10,1516.85,1713200,0,0
2020-07-20,1515.00,1569.01,1502.70,1563.84,1508300,0,0


In [3]:
hist["Change"]=hist["Close"]-hist["Open"]

In [4]:
hist.loc[hist['Open']>= hist['Close'], 'Result'] = 0
hist.loc[hist['Open']< hist['Close'], 'Result'] = 1

In [5]:
hist = hist.drop(columns =['Open','High','Low','Close','Volume','Dividends','Stock Splits','Change'])

In [6]:
hist.reset_index()

,Date,Result
0,2014-07-22,1.0
1,2014-07-23,1.0
2,2014-07-24,0.0
3,2014-07-25,0.0
4,2014-07-28,1.0
...,...,...
1507,2020-07-16,1.0
1508,2020-07-17,0.0
1509,2020-07-20,1.0
1510,2020-07-21,0.0


In [7]:
news7daylag_nyt_reuter_technewsworld_df = pd.read_csv("data/news7daylag_nyt_reuters_technewsworld_data.csv")

In [8]:
news7daylag_nyt_reuter_technewsworld_df.columns=['Date','all_news']

In [9]:
news7daylag_nyt_reuter_technewsworld_df['Date'] = pd.to_datetime(news7daylag_nyt_reuter_technewsworld_df['Date'], errors='coerce')

In [10]:
stocknews_df = pd.merge(news7daylag_nyt_reuter_technewsworld_df,hist,how="inner",on="Date")

In [11]:
stocknews_df = stocknews_df[['Date','Result','all_news']]

In [12]:
stocknews_df

,Date,Result,all_news
0,2014-07-22,1.0,new regulations could lead to the closing of t...
1,2014-07-23,1.0,a dropped device s chances of emerging with it...
2,2014-07-24,0.0,dr jordan metzl is a sports medicine physicia...
3,2014-07-25,0.0,to reflect the diversity of experiences in the...
4,2014-07-28,1.0,weird al yankovic above topped the albums ch...
...,...,...,...
1504,2020-07-13,0.0,her lawyer who issued a statement on the day ...
1505,2020-07-14,1.0,the trump administration said it would no long...
1506,2020-07-15,0.0,a fact finding team will have no power to subp...
1507,2020-07-16,1.0,terence a monahan new york s highest ranking...


In [13]:
stocknews_df.loc[(stocknews_df['Result']!=0) & (stocknews_df['Result']!=1)]

,Date,Result,all_news


In [14]:
stocknews_df = stocknews_df.dropna()

In [15]:
stocknews_df.dtypes

Date        datetime64[ns]
Result             float64
all_news            object
dtype: object

In [16]:
#stocknews_df.to_csv('data/facebook_nytreuterstechnewsworld7lag_cleanedmerged.csv',index=False)

In [17]:
train = stocknews_df[stocknews_df['Date']<'2018-07-17']
test = stocknews_df[stocknews_df['Date']>'2018-07-16']

In [18]:
data=train.iloc[:,2]

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [20]:
countvector = CountVectorizer(ngram_range=(2,2))
traindataset = countvector.fit_transform(data)

In [21]:
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(traindataset,train['Result'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [22]:
test_transform = []
for row in range(0, len(test.index)):
    test_transform.append(''.join(str(x) for x in test.iloc[row,2]))
#test_transform[0]

In [23]:
test_dataset = countvector.transform(test_transform)
predictions = randomclassifier.predict(test_dataset)

In [24]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [25]:
matrix = confusion_matrix(test['Result'], predictions)
print(matrix)
tp, fn, fp, tn = confusion_matrix(test['Result'],predictions,labels=[1,0]).reshape(-1)
print(tp, fn, fp, tn)
score = accuracy_score(test['Result'],predictions)
print(score)
report = classification_report(test['Result'],predictions)
print(report)

[[ 27 219]
 [ 32 227]]
227 32 219 27
0.502970297029703
              precision    recall  f1-score   support

         0.0       0.46      0.11      0.18       246
         1.0       0.51      0.88      0.64       259

    accuracy                           0.50       505
   macro avg       0.48      0.49      0.41       505
weighted avg       0.48      0.50      0.42       505



In [26]:
results_df = pd.DataFrame({'Date': test['Date'], 'Label': test['Result'], 'Prediction': predictions})

In [27]:
results_df

,Date,Label,Prediction
1004,2018-07-17,1.0,0.0
1005,2018-07-18,1.0,0.0
1006,2018-07-19,0.0,0.0
1007,2018-07-20,0.0,1.0
1008,2018-07-23,1.0,0.0
...,...,...,...
1504,2020-07-13,0.0,1.0
1505,2020-07-14,1.0,0.0
1506,2020-07-15,0.0,0.0
1507,2020-07-16,1.0,0.0


In [28]:
results_df.to_csv('data/google_predictions.csv',index=False)

In [29]:
confusion_matrix_df = pd.DataFrame({'True Positive': [tp], 'True Negative': [tn], 'False Positive': [fp], 'False Negative': [fn], 'Model Accuracy': [score]})

In [30]:
confusion_matrix_df

,True Positive,True Negative,False Positive,False Negative,Model Accuracy
0,227,27,219,32,0.50297


In [31]:
confusion_matrix_df.to_csv('data/google_confusion_matrix.csv',index=False)